In [1]:
import swmmAPI as swmm

headers = [
    '[TITLE]',
    '[OPTIONS]',
    '[EVAPORATION]',
    '[RAINGAGES]',
    '[SUBCATCHMENTS]',
    '[SUBAREAS]',
    '[INFILTRATION]',
    '[JUNCTIONS]',
    '[OUTFALLS]',
    '[STORAGE]',
    '[CONDUITS]',
    '[PUMPS]',
    '[ORIFICES]',
    '[WEIRS]',
    '[XSECTIONS]',
    '[LOSSES]',
    '[CONTROLS]',
    '[INFLOWS]',
    '[DWF]',
    '[HYDROGRAPHS]',
    '[RDII]',
    '[CURVES]',
    '[TIMESERIES]',
    '[PATTERNS]',
    '[REPORT]',
    '[TAGS]',
    '[MAP]',
    '[COORDINATES]',
    '[VERTICES]',
    '[POLYGONS]',
    '[SYMBOLS]',
    '[PROFILES]'
]

inpF = 'NoWestside_withISDs_RainEvents.inp'

sections = swmm.make_sections(inpF,headers)
conduitDict,junctionDict,storageDict,subcatchmentDict,outfallDict,orificeDict,curvesDict,pumpDict,optionsDict = swmm.prep_dicts(sections)